In [14]:
import pandas as pd
import numpy as np
import multiprocessing
import subprocess
import random
import pickle
import shutil
import glob
import math
import csv
import sys
import os
import re

from pybedtools import BedTool

import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
from matplotlib_venn import venn2
from matplotlib_venn import venn3
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 24
%matplotlib inline

gff3Cols=["seqid","source","type","start","end","score","strand","phase","attributes"]

In [15]:
base_dir = "./"
out_dir = base_dir+"step6/"
soft_dir = base_dir+"soft/"

chess3_gtf_fname = bae_dir+"step5/chess.gtf"

data_dir = base_dir+"data/"
gencode_gtf_fname = data_dir+"latest_gtfs/gencode.primary.gtf"
refseq_gtf_fname = data_dir+"latest_gtfs/refseq.primary.gtf"
mane_gtf_fname = data_dir+"latest_gtfs/MANE.primary.gtf"
M27_gtf_fname = data_dir+"latest_gtfs/27M.primary.gtf"
chess2_gtf_fname = data_dir+"latest_gtfs/chess2.gtf"

ref_fasta_fname = "hg38_p12_ucsc.no_alts.no_fixs.fa"
remove_gtf = data_dir+"chess_subset.gtf"

gffcompare_bin = "gffcompare"

num_threads = 40

In [16]:
def get_by_tag(gtf_fname,kvs,out_fp): # kvs is a dict of individual keys to sets of passing labels
    with open(gtf_fname,"r") as inFP:
        write_tx = False
        for line in inFP:
            lcs = line.strip().split("\t")
            if lcs[2]=="transcript":
                write_tx = False
                atts = [(x.split(" \"",1)[0].strip(),x.split(" \"",1)[1].rstrip("\"")) for x in lcs[8].split(";")]
                for ak,av in atts:
                    if ak in kvs:
                        if av in kvs[ak]:
                            write_tx=True
                            break
            if write_tx:
                out_fp.write(line)

In [17]:
# select selenocysteine (gencode: tag "seleno" - 79 txs)

gencode_passing_kvs = {"tag":{"seleno"}}
out_gencode_fname = out_dir+"must_include.gencode.gtf"
out_gencode_fp = open(out_gencode_fname,"w+")
get_by_tag(gencode_gtf_fname,gencode_passing_kvs,out_gencode_fp)
out_gencode_fp.close()

refseq_passing_kvs = {"tag":{"MANE Select","RefSeq Select"},
                      "gbkey":{"V_region", "V_segment", "D_segment", "J_segment", "C_region",
                               "tRNA", "rRNA", "snRNA", "snoRNA", "tmRNA","Y_RNA"},
                      "gene_biotype":{"V_region", "V_segment", "D_segment", "J_segment", "C_region",
                                      "tRNA", "rRNA", "snRNA", "snoRNA", "tmRNA","Y_RNA"}}
out_refseq_fname = out_dir+"must_include.refseq.gtf"
out_refseq_fp = open(out_refseq_fname,"w+")
get_by_tag(refseq_gtf_fname,refseq_passing_kvs,out_refseq_fp)
out_refseq_fp.close()

In [18]:
def load_gid_tid(gtf_fname): # load dictionary of gid to max_tid
    gtf = pd.read_csv(gtf_fname,sep="\t",comment="#",names=gff3Cols)
    gtf = gtf[gtf["type"]=="transcript"].reset_index(drop=True)
    gtf["tmp"] = gtf["attributes"].str.split("transcript_id \"",expand=True,n=1)[1].str.split("\"",expand=True,n=1)[0]
    gtf["gid"] = gtf["tmp"].str.split(".",expand=True,n=2)[1].astype(int)
    gtf["tid"] = gtf["tmp"].str.split(".",expand=True,n=2)[2].astype(int)
    gtf=gtf[["gid","tid"]]
    ggtf = gtf.groupby(by="gid").max().reset_index()
    gdict = ggtf.set_index('gid').to_dict()['tid']
    return gdict

def load_tid_gid(gtf_fname): # load dictionary of tid to gid
    gtf = pd.read_csv(gtf_fname,sep="\t",comment="#",names=gff3Cols)
    gtf = gtf[gtf["type"]=="transcript"].reset_index(drop=True)
    tdict = dict()
    gtf["tid"] = gtf["attributes"].str.split("transcript_id \"",expand=True,n=1)[1].str.split("\"",expand=True,n=1)[0]
    gtf["gid"] = gtf["attributes"].str.split("gene_id \"",expand=True,n=1)[1].str.split("\"",expand=True,n=1)[0]
    gtf=gtf[["gid","tid"]]
    tdict = gtf.set_index('tid').to_dict()['gid']
    return tdict

def load_tid_rec(gtf_fname): # load dictionary of gtf to full transcript (transcript,exons,cds) record
    recs = dict()
    with open(gtf_fname,"r") as inFP:
        for line in inFP:
            if len(line.strip())==0:
                continue
            lcs = line.strip().split("\t")
            tid = lcs[8].split("transcript_id \"",1)[1].split("\"",1)[0]
            if lcs[2]=="transcript":
                assert tid not in recs,"duplicate tids: "+tid
                recs[tid]=line
            else:
                assert tid in recs,"tid not found: "+tid
                recs[tid]+=line
                
    return recs

def add_from_tmap(ref_gtf_fname,qry_gtf_fname,tmap,out_fname,kv2add,qtmap):
    outFP = open(out_fname,"w+")
    
    ref_gdict = load_gid_tid(ref_gtf_fname)
    next_gid = max(ref_gdict)+1
    
    
    # load qry into dict (tid to gtf record)
    qry_recs = load_tid_rec(qry_gtf_fname)
    qry_tdict = load_tid_gid(qry_gtf_fname)  
    
    # parse tmap and decide from the code what to do
    pass_codes = set(["=","c","k","j","m","n"])

    tmp = pd.read_csv(tmap,sep="\t",usecols=[0,1,2,3,4])
    
    loaded_tids = set() # keeps track of which tids have been used to not overwrite with reference

    for qgid,grp in tmp.groupby(by="qry_gene_id"):
        rgids = set(list(grp["ref_gene_id"]))
        assert len(rgids)==1,"multiple group gids: "+qgid+"\t"+str(rgids)

        known = len(pass_codes.intersection(set(list(grp["class_code"]))))>0 # at least some codes in the passing codes - add everything to the known gene

        rgid = ""
        next_tid = 0
        if not known: # novel
            rgid = next_gid
            next_gid+=1
        else:
            assert not rgid=="-","missing reference gid assigned"
            rgid = int(list(rgids)[0].split(".")[1])

        for idx,row in grp.iterrows():
            qtid = row["qry_id"]
            new_tid = ""
            new_gid = ""
            
            if known:
                code = row["class_code"]
                if code=="=": # already exists
                    new_tid = row["ref_id"]
                    new_gid = row["ref_gene_id"]
                else: # add to known gene as a new transcript and increment max_tid
                    # check if exists in the map
                    if row["qry_id"] in qtmap:
                        new_tid = qtmap[row["qry_id"]]
                        new_gid = "CHS."+new_tid.split(".")[1]
                        gid_num = int(new_tid.split(".")[1])
                        tid_num = int(new_tid.split(".")[2])
#                         assert gid_num==rgid,"gene ids do not match: "+str(gid_num)+" "+str(rgid)+" "+",".join(rgids)+" "+new_tid
                        assert new_tid not in loaded_tids,"tid already found"
                    else:
                        new_tid = "CHS."+str(rgid)+"."+str(ref_gdict[rgid]+1)
                        new_gid = "CHS."+str(rgid)
                        ref_gdict[rgid]+=1
                
            else:
                # check if exists in the map
                if row["qry_id"] in qtmap:
                    new_tid = qtmap[row["qry_id"]]
                    new_gid = "CHS."+new_tid.split(".")[1]
                else:
                    new_tid = "CHS."+str(rgid)+"."+str(next_tid)
                    new_gid = "CHS."+str(rgid)
                    next_tid+=1
            
            if new_tid in loaded_tids:
                print("duplicate ID: "+new_tid)
                continue
            
            loaded_tids.add(new_tid)
            rec=qry_recs[qtid]
            for line in rec.split("\n"):
                if len(line.strip())==0:
                    continue
                lcs = line.strip().split("\t")
                atts = [(x.split(" \"")[0].strip(),x.split("\"")[1]) for x in lcs[8].strip(";").split(";")]
                
                for k,v in atts:
                    if k==kv2add[0]:
                        kv2add[1]+=","+v
                        
                atts = [x for x in atts if x[0] not in ["transcript_id","gene_id",kv2add[0]]]
                        
                atts = "transcript_id \""+new_tid+\
                       "\"; gene_id \""+new_gid+\
                       "\"; old_transcript_id \""+row["qry_id"]+\
                       "\"; old_gene_id \""+row["qry_gene_id"]+\
                       "\"; "+kv2add[0]+" \""+kv2add[1]+\
                       "\"; "+"; ".join([x[0]+" \""+x[1]+"\"" for x in atts])+";"

                lcs[8] = atts
                outFP.write("\t".join(lcs)+"\n")
                
                
    # add the rest of the reference
    with open(ref_gtf_fname,"r") as inFP:
        for line in inFP:
            if len(line.strip())==0:
                continue
            lcs = line.strip().split("\t")
            tid = lcs[8].split("transcript_id \"",1)[1].split("\"",1)[0]
            if not tid in loaded_tids:
                outFP.write(line)
                
    outFP.close()

In [19]:
# load a map of "=" codes
def load_trmap(tr_fname):
    tqmap = dict()
    qtmap = dict()
    with open(tr_fname,"r") as inFP:
        for line in inFP:
            if line[0]=="#":
                continue
            lcs = line.strip("\n").split("\t")

            code = lcs[3]
            if code == "=":
                rtid = lcs[2].split("|")[1]
                qtid = lcs[4].split("|")[1]
                tqmap.setdefault(rtid,list())
                tqmap[rtid].append(qtid)
                
                qtmap.setdefault(qtid,list())
                qtmap[qtid].append(rtid)
                
    for k,v in tqmap.items():
        tqmap[k] = ",".join(v)
        
    for k,v in qtmap.items():
        qtmap[k] = ",".join(v)
        
    return tqmap,qtmap

In [ ]:
# run gffcompare between the must include and chess
if not os.path.exists(out_dir+"tmp_gen/"):
    os.makedirs(out_dir+"tmp_gen/")
gffcmp_cmd = [gffcompare_bin,"--no-merge",
              "-o", out_dir+"tmp_gen/cmp",
              "-r", chess3_gtf_fname,
              out_gencode_fname]
print(" ".join(gffcmp_cmd))
subprocess.call(gffcmp_cmd)
shutil.move(out_dir+"cmp.must_include.gencode.gtf.refmap",out_dir+"tmp_gen/cmp.refmap")
shutil.move(out_dir+"cmp.must_include.gencode.gtf.tmap",out_dir+"tmp_gen/cmp.tmap")

gffcmp_cmd = [gffcompare_bin,"--no-merge",
              "-o", out_dir+"tmp_gen/cmp_id",
              "-r", chess2_gtf_fname,
              out_gencode_fname]
print(" ".join(gffcmp_cmd))
subprocess.call(gffcmp_cmd)
tqmap,qtmap = load_trmap(out_dir+"tmp_gen/cmp_id.tracking")

add_from_tmap(chess3_gtf_fname,out_gencode_fname,out_dir+"tmp_gen/cmp.tmap",out_dir+"tmp.ref_gen.gtf",["step6","GENCODE"],qtmap)

In [ ]:
# repeat the same procedure with the new set of transcripts from refseq
if not os.path.exists(out_dir+"tmp_ref/"):
    os.makedirs(out_dir+"tmp_ref/")
gffcmp_cmd = [gffcompare_bin,"--no-merge",
              "-o", out_dir+"tmp_ref/cmp",
              "-r", out_dir+"tmp.ref_gen.gtf",
              out_refseq_fname]
print(" ".join(gffcmp_cmd))
subprocess.call(gffcmp_cmd)
shutil.move(out_dir+"cmp.must_include.refseq.gtf.refmap",out_dir+"tmp_ref/cmp.refmap")
shutil.move(out_dir+"cmp.must_include.refseq.gtf.tmap",out_dir+"tmp_ref/cmp.tmap")

gffcmp_cmd = [gffcompare_bin,"--no-merge",
              "-o", out_dir+"tmp_ref/cmp_id",
              "-r", chess2_gtf_fname,
              out_refseq_fname]
print(" ".join(gffcmp_cmd))
subprocess.call(gffcmp_cmd)
tqmap,qtmap = load_trmap(out_dir+"tmp_ref/cmp_id.tracking")

add_from_tmap(out_dir+"tmp.ref_gen.gtf",out_refseq_fname,out_dir+"tmp_ref/cmp.tmap",out_dir+"tmp.ref_gen_refseq.gtf",["step6","RefSeq"],qtmap)

In [ ]:
# repeat the same procedure with the new set of transcripts from mane
if not os.path.exists(out_dir+"tmp_mane/"):
    os.makedirs(out_dir+"tmp_mane/")
gffcmp_cmd = [gffcompare_bin,"--no-merge",
              "-o", out_dir+"tmp_mane/cmp",
              "-r", out_dir+"tmp.ref_gen_refseq.gtf",
              mane_gtf_fname]
print(" ".join(gffcmp_cmd))
subprocess.call(gffcmp_cmd)
mane_tmap = "/".join(mane_gtf_fname.split("/")[:-1])+"/cmp."+mane_gtf_fname.split("/")[-1]+".tmap"
mane_refmap = "/".join(mane_gtf_fname.split("/")[:-1])+"/cmp."+mane_gtf_fname.split("/")[-1]+".refmap"
shutil.move(mane_refmap,out_dir+"tmp_mane/cmp.refmap")
shutil.move(mane_tmap,out_dir+"tmp_mane/cmp.tmap")

gffcmp_cmd = [gffcompare_bin,"--no-merge",
              "-o", out_dir+"tmp_mane/cmp_id",
              "-r", chess2_gtf_fname,
              mane_gtf_fname]
print(" ".join(gffcmp_cmd))
subprocess.call(gffcmp_cmd)
tqmap,qtmap = load_trmap(out_dir+"tmp_mane/cmp_id.tracking")

add_from_tmap(out_dir+"tmp.ref_gen_refseq.gtf",mane_gtf_fname,out_dir+"tmp_mane/cmp.tmap",out_dir+"tmp.ref_gen_refseq_mane.gtf",["step6","MANE"],qtmap)

In [ ]:
# lastly, we need to add ALL-ids for all transcripts based on the comparison to the 27M set

gffcmp_cmd = ["gffcompare",
              "-o",out_dir+"27McmpStep6",
              "-r",out_dir+"tmp.ref_gen_refseq_mane.gtf",
              M27_gtf_fname]
print(" ".join(gffcmp_cmd))
# subprocess.call(gffcmp_cmd)

In [24]:
# load a map of IDs
ass_id_map = dict()
with open(out_dir+"27McmpStep6.tracking","r") as inFP:
    for line in inFP:
        lcs = line.split("\t")
        code = lcs[3]
        if code == "=":
            ass_id = lcs[4].split("|")[1]
            chs_id = lcs[2].split("|")[1]
            ass_id_map[chs_id] = ass_id

In [25]:
with open(out_dir+"chess.gtf","w+") as outFP:
    with open(out_dir+"tmp.ref_gen_refseq_mane.gtf","r") as inFP:
        for line in inFP:
            if len(line.strip())==0:
                continue
            lcs = line.strip().split("\t")
            if len(lcs)<9:
                continue
            if lcs[2]=="transcript":
                # add attribute with the original ID
                tid = lcs[8].split("transcript_id \"",1)[1].split("\"",1)[0]
                ass_id = "none"
                if tid in ass_id_map:
                    ass_id = ass_id_map[tid]
                lcs[8] = lcs[8].rstrip().rstrip(";")+"; assembly_id \""+ass_id+"\";"
            line = "\t".join(lcs).rstrip()+"\n"
            outFP.write(line)